In [24]:
# pip install qdrant-client openai tqdm numpy

import os, json, re, unicodedata, uuid
from typing import List, Dict, Any, Optional
from tqdm import tqdm
import numpy as np
from qdrant_client import QdrantClient, models
from openai import OpenAI
import uuid

# ========= env =========
client = OpenAI()  # OPENAI_API_KEY を .env 等で
qdrant = QdrantClient(":memory:")

N = 10
EMBED_MODEL = "text-embedding-3-large"  # 3072-d
INPUT_DATA = f"../data/sample/sample_{N}.json"
QDRANT_COLLECTION = f"fashion_{N}_collection"
MODEL = "gpt-4o-mini"

In [25]:
# ========= utils =========
def load_items(path: str) -> List[Dict[str, Any]]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    if isinstance(data, list): return data
    for k in ("items", "results"):
        if k in data: return data[k]
    raise ValueError("items/results not found")

def get_image_url(item: Dict[str, Any]) -> Optional[str]:
    imgs = item.get("images") or []
    for im in imgs:
        for key in ("hi_res", "large", "thumb"):
            if im.get(key): return im[key]
    return None

def norm(s: str) -> str:
    if not isinstance(s, str): return ""
    s = unicodedata.normalize("NFKC", s).strip()
    return re.sub(r"\s+", " ", s)

def build_text_context(item: Dict[str, Any]) -> str:
    parts = [item.get("title",""), item.get("store","")]
    if isinstance(item.get("features"), list): parts.extend(item["features"])
    if isinstance(item.get("description"), list): parts.extend(item["description"])
    for k in ("caption","image_caption","caption_image_only","caption_with_text"):
        if item.get(k): parts.append(item[k])
    det = item.get("details", {})
    if isinstance(det, dict): parts.extend([f"{k}: {v}" for k,v in det.items()])
    return norm(" ".join(p for p in parts if p))



In [26]:
# ========= caption (image + text) =========
SYSTEM_PROMPT = """
You are an e-commerce image captioning assistant.
Always write in English. Produce 1–2 factual sentences only.
Describe what is visible or explicitly provided; do not speculate.
Brand only if visible in the image or explicitly provided; otherwise omit.
Prefer concrete attributes: category, color, material, silhouette/fit, notable details.
"""
USER_PROMPT = """
Write one concise factual caption (1–2 sentences) for this product image using visual content and the context below.
Do NOT contradict the image.

Context:
Title: {title}
Store: {store}
Features: {features}
"""

def generate_caption(image_url: str, item: Dict[str, Any],
                     model="gpt-4o-mini", temperature=0.2, max_tokens=120) -> str:
    feats = item.get("features", [])
    user = USER_PROMPT.format(
        title=item.get("title",""), store=item.get("store",""),
        features="; ".join(feats) if isinstance(feats, list) else ""
    ).strip()
    content = [{"type": "text", "text": user},
               {"type": "image_url", "image_url": {"url": image_url}}]
    try:
        r = client.chat.completions.create(
            model=model,
            messages=[{"role":"system","content":SYSTEM_PROMPT.strip()},
                      {"role":"user","content":content}],
            temperature=temperature, max_tokens=max_tokens
        )
        return (r.choices[0].message.content or "").strip()
    except Exception as e:
        print("[caption error]", e)
        return ""

# ========= embeddings =========
def embed_texts(texts: List[str], batch=128) -> np.ndarray:
    vecs = []
    for i in range(0, len(texts), batch):
        ch = texts[i:i+batch]
        r = client.embeddings.create(model=EMBED_MODEL, input=ch)
        vecs.extend([d.embedding for d in r.data])
    return np.asarray(vecs, dtype=np.float32)

# ========= BM25 sparse encoder (doc/query) =========
TOKEN_RE = re.compile(r"[a-z0-9]+")

def tok_en(s: str) -> List[str]:
    return TOKEN_RE.findall(norm(s).lower())

class BM25Sparse:
    """
    Document側: values = saturation(tf, dl, avgdl, k1, b)
    Query側   : values = idf
    dot(query, doc) = BM25 score (qtf省略版)
    """
    def __init__(self, k1=0.9, b=0.4):
        self.k1 = k1
        self.b = b
        self.vocab = {}  # token -> id
        self.df = None
        self.idf = None
        self.N = 0
        self.avgdl = 0.0

    def fit(self, docs: List[str]):
        self.N = len(docs)
        lengths = []
        df_counter = {}
        for d in docs:
            toks = tok_en(d)
            lengths.append(len(toks))
            for t in set(toks):
                df_counter[t] = df_counter.get(t, 0) + 1
        self.avgdl = float(np.mean(lengths) if lengths else 0.0)
        self.vocab = {t:i for i,t in enumerate(sorted(df_counter.keys()))}
        self.df = np.zeros(len(self.vocab), dtype=np.int32)
        for t,i in self.vocab.items():
            self.df[i] = df_counter[t]
        # BM25 idf (Robertson/Sparck Jones)
        self.idf = np.log((self.N - self.df + 0.5) / (self.df + 0.5) + 1.0).astype(np.float32)

    def transform_doc(self, text: str):
        toks = tok_en(text)
        if not toks or not self.vocab: return [], []
        dl = len(toks) if len(toks)>0 else 1
        tf = {}
        for t in toks:
            i = self.vocab.get(t)
            if i is not None:
                tf[i] = tf.get(i, 0) + 1
        if not tf: return [], []
        idx = np.fromiter(tf.keys(), dtype=np.int32)
        tfv = np.fromiter(tf.values(), dtype=np.float32)
        k1, b = self.k1, self.b
        denom = tfv + k1*(1 - b + b * dl / max(self.avgdl, 1e-9))
        vals = (tfv * (k1 + 1.0)) / denom
        # 正規化（任意。コサインっぽく安定）
        n = np.linalg.norm(vals)
        if n > 0: vals = vals / n
        return idx.tolist(), vals.astype(np.float32).tolist()

    def transform_query(self, text: str):
        toks = tok_en(text)
        ids, vals = [], []
        for t in set(toks):
            i = self.vocab.get(t)
            if i is not None:
                ids.append(i); vals.append(float(self.idf[i]))
        return ids, vals

# ========= Qdrant collection (dense + sparse) =========
def create_collection(collection: str, dim=3072):
    qdrant.recreate_collection(
        collection_name=collection,
        vectors_config={"text-dense": models.VectorParams(size=dim, distance=models.Distance.COSINE)},
        sparse_vectors_config={"text-sparse": models.SparseVectorParams(index=models.SparseIndexParams(on_disk=False))}
    )
    print(f"[Qdrant] ready: {collection} (dense={dim}, sparse=BM25)")


In [ ]:
import uuid, json
from tqdm import tqdm
from qdrant_client import models

def ingest_json_to_qdrant_hybrid(
    data_path: str,
    collection: str,
    caption_model: str = "gpt-4o-mini",
    batch_upsert: int = 128,
    overwrite_collection: bool = True
):
    items = load_items(data_path)

    prepared = []
    for it in tqdm(items, desc="Captioning"):
        img = get_image_url(it)
        if not img:
            continue

        cap = generate_caption(img, it, model=caption_model)
        base = build_text_context(it)
        combined = norm(f"{base} {cap}".strip())
        prepared.append({
            "id": it.get("parent_asin") or it.get("asin") or str(uuid.uuid4()),
            "combined_text": combined,
            "payload": it,
            "_caption": cap,
            "_image_url": img
        })

    if not prepared:
        raise RuntimeError("No items to ingest")

    # Dense embeddings
    texts = [p["combined_text"] for p in prepared]
    dense = embed_texts(texts)

    # BM25 sparse
    bm25 = BM25Sparse(k1=0.9, b=0.4)
    bm25.fit(texts)  # df/avgdl/N 計算

    # Qdrant collection
    if overwrite_collection:
        create_collection(collection, dim=dense.shape[1])

    # --- Upsert points ---
    points = []
    for i, p in enumerate(prepared):
        idx, vals = bm25.transform_doc(p["combined_text"])

        try:
            uid = uuid.UUID(str(p["id"]))
        except Exception:
            uid = uuid.uuid5(uuid.NAMESPACE_DNS, str(p["id"]))

        points.append(
            models.PointStruct(
                id=str(uid),
                vector={
                    "text-dense": dense[i].tolist(),
                    "text-sparse": models.SparseVector(indices=idx, values=vals),
                },
                payload={
                    **p["payload"],
                    "_combined_text": p["combined_text"],
                    "_caption": p["_caption"],
                    "_image_url": p["_image_url"],
                },
            )
        )

        if len(points) >= batch_upsert:
            qdrant.upsert(collection_name=collection, points=points)
            points = []

    if points:
        qdrant.upsert(collection_name=collection, points=points)

    # --- 永続化: BM25統計メタをQdrantに保存 ---
    bm25_meta = {
        "_bm25_meta": True,
        "N": bm25.N,
        "avgdl": bm25.avgdl,
        "df": bm25.df,
        "tokenizer": "basic_v1"
    }
    qdrant.upsert(
        collection_name=collection,
        points=[
            models.PointStruct(id="__bm25_meta__", payload=bm25_meta)
        ]
    )

    # --- 検索時にメモリ保持（任意） ---
    global _BM25_INDEXER_
    _BM25_INDEXER_ = bm25

    print(f"[Done] {len(prepared)} points ingested into '{collection}' (BM25 meta stored)")


In [28]:
# 1) 取り込み
ingest_json_to_qdrant_hybrid(
    data_path=INPUT_DATA,
    collection=QDRANT_COLLECTION,
    caption_model=MODEL,
    overwrite_collection=True
)

Captioning: 100%|██████████| 10/10 [00:28<00:00,  2.90s/it]


[Qdrant] ready: fashion_10_collection (dense=3072, sparse=BM25)
[Done] 10 points ingested into 'fashion_10_collection'


C:\Users\yagik\AppData\Local\Temp\ipykernel_2528\3447557462.py:118: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


In [ ]:
from qdrant_client import models

def load_bm25_from_qdrant(collection: str):
    pts = qdrant.retrieve(collection_name=collection, ids=["__bm25_meta__"])
    if not pts or not pts[0].payload.get("_bm25_meta"):
        raise RuntimeError("BM25 meta not found in Qdrant")

    meta = pts[0].payload
    bm25 = BM25Sparse(k1=0.9, b=0.4)
    bm25.N = meta["N"]
    bm25.avgdl = meta["avgdl"]
    bm25.df = {k: int(v) for k, v in meta["df"].items()}
    bm25.tokenizer_version = meta.get("tokenizer", "basic_v1")
    return bm25


def hybrid_search(query: str, collection: str, top_k: int = 10, prefetch_k: int = 50):
    """
    Qdrant内に永続化されたBM25統計を利用して、Dense + Sparseハイブリッド検索を実行。
    ローカルpickle不要。
    """
    # --- Dense embedding ---
    q_dense = embed_texts([norm(query)])[0].tolist()

    # --- Sparse (BM25) ---
    _BM25_INDEXER_ = load_bm25_from_qdrant(collection)

    q_idx, q_vals = _BM25_INDEXER_.transform_query(query)

    # --- Hybrid Search (RRF融合) ---
    res = qdrant.query_points(
        collection_name=collection,
        prefetch=[
            models.Prefetch(
                query=models.SparseVector(indices=q_idx, values=q_vals),
                using="text-sparse",
                limit=prefetch_k,
            ),
            models.Prefetch(
                query=q_dense,
                using="text-dense",
                limit=prefetch_k,
            ),
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),  # Reciprocal Rank Fusion
        with_payload=True,
        limit=top_k,
    )

    # --- 整形して返す ---
    return [
        {
            "id": p.id,
            "score": p.score,
            "title": p.payload.get("title"),
            "_caption": p.payload.get("_caption"),
            "_image_url": p.payload.get("_image_url"),
        }
        for p in res.points
    ]


In [30]:
# 2) 検索（サーバー側RRFでdense+sparse融合）
hits = hybrid_search(
    "lightweight summer dress for the beach", 
    QDRANT_COLLECTION, 
    top_k=10)
import pandas as pd
pd.DataFrame(hits)

,id,score,title,_caption,_image_url
0,50557528-baeb-53e6-84b9-248c941af5db,0.625000,HISITOSA Men's Dress Belt Black Leather Revers...,HISITOSA Men's Dress Belt features a reversibl...,https://m.media-amazon.com/images/I/61ukXBH1Ld...
1,00217fb0-f735-5732-addc-4335cd6d6067,0.611111,Women's Japanese Traditional Kimono Costume An...,Women's black kimono-style bathrobe featuring ...,https://m.media-amazon.com/images/I/81OOA6+NUV...
2,839b9f35-3d13-5cab-8476-d189c11d3aec,0.583333,Bimba Women Pink Anarkali Kurti Long Flaired R...,Bimba Women’s Pink Anarkali Kurti features a l...,https://m.media-amazon.com/images/I/71bdfM3QQI...
3,e8e9093b-e20a-5d3d-b91e-05daccbf59cb,0.424242,Popular Designed Peace Sign With Dragonfly Art...,This round mouse pad features a vibrant peace ...,https://m.media-amazon.com/images/I/61AewVYI9y...
4,906b4ab2-6d15-5060-b2aa-ab3a95a13c42,0.333333,PajamaMania Junior's Hooded Fleece Short Robe ...,Cozy up in the PajamaMania Junior's Hooded Fle...,https://m.media-amazon.com/images/I/81rPqnsgXD...
5,92f82164-821f-51b8-856c-666aae841be0,0.325000,Black classic fashion casual business ladies b...,This black classic ladies' belt is made from 1...,https://m.media-amazon.com/images/I/51YX8+mYiK...
6,e62d499d-d353-5bc9-be7f-bdc22ece9aee,0.300000,Golf Shoe Grabber White,The Golf Shoe Grabber in white is a versatile ...,https://m.media-amazon.com/images/I/71lm+GoaOX...
7,cef824ca-9904-5188-a776-d2cca78698a1,0.253968,Callahan Auto Parts Shirt - Distressed Vintage...,The Callahan Auto Parts Shirt features a distr...,https://m.media-amazon.com/images/I/71eAFVjNTK...
8,723198b1-66c2-5b7e-94fd-fbbe92db9aa8,0.250000,iDeesse Women's Plus Size Long Sleeve Cold Sho...,iDeesse Women's Plus Size Long Sleeve Cold Sho...,https://m.media-amazon.com/images/I/618CDV89nJ...
9,34b1d4c7-62df-52e6-b95b-999cc75b8320,0.142857,Trunk Candy New Monarchy Standard Logo Men's M...,Men's modern fit tri-blend t-shirt in vintage ...,https://m.media-amazon.com/images/I/71O+cIuf7V...
